## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH09-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH09-Models


모델과 프롬프트를 생성합니다


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [4]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 조선민주주의인민공화국으로 구성되어 있는 나라이다. 대한민국은 수도가 서울에 위치하고 있으며, 고도의 경제력과 기술력을 보유한 선진국가로 발전해 왔다. 한류와 K-pop 등의 문화 콘텐츠로 세계적으로 유명하며, 전통문화와 현대문화가 조화를 이루고 있다. 또한, 한반도 북쪽에 위치한 조선민주주의인민공화국은 사회주의 체제를 갖추고 있으며, 국내외에서의 관계가 긴장되어 있는 상황이다. 두 나라는 공통된 역사와 문화를 공유하고 있지만, 이념과 체제 차이로 인해 국가 간 갈등과 분쟁이 계속되고 있다.
CPU times: user 73.5 ms, sys: 28.7 ms, total: 102 ms
Wall time: 3.02 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [5]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 조선민주주의인민공화국으로 구성된 한반도의 국가이다. 서울이라는 수도를 중심으로 다양한 문화와 역사를 자랑하며, 전통적인 한옥과 현대적인 건축물이 공존하는 독특한 모습을 보여준다. 한국은 전통적인 한복을 비롯한 다양한 문화유산과 한류 문화로 유명하며, 한글이라는 독특한 문자체계를 사용하고 있다. 또한, 한반도의 자연환경은 다양한 산악지대와 바다로 둘러싸여 있어 자연경관이 아름다운 편이다. 한국은 선진화된 IT 기술과 경제력을 바탕으로 세계적으로 각광받고 있으며, 한반도 주변 국가와의 관계도 중요한 역할을 하고 있다.
CPU times: user 291 ms, sys: 48.2 ms, total: 340 ms
Wall time: 4.2 s


In [6]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 조선민주주의인민공화국으로 구성된 한반도의 국가이다. 서울이라는 수도를 중심으로 다양한 문화와 역사를 자랑하며, 전통적인 한옥과 현대적인 건축물이 공존하는 독특한 모습을 보여준다. 한국은 전통적인 한복을 비롯한 다양한 문화유산과 한류 문화로 유명하며, 한글이라는 독특한 문자체계를 사용하고 있다. 또한, 한반도의 자연환경은 다양한 산악지대와 바다로 둘러싸여 있어 자연경관이 아름다운 편이다. 한국은 선진화된 IT 기술과 경제력을 바탕으로 세계적으로 각광받고 있으며, 한반도 주변 국가와의 관계도 중요한 역할을 하고 있다.
CPU times: user 5.78 ms, sys: 979 μs, total: 6.76 ms
Wall time: 15.3 ms


## SQLite Cache


In [7]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [10]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "미국"})
print(response.content)

미국은 북아메리카에 위치한 연방 공화국으로, 50개의 주로 이루어져 있다. 세계에서 가장 강대한 국가 중 하나로 경제적, 군사적, 문화적으로 세계적 영향력을 가지고 있다. 원칙적으로 민주주의를 지향하며, 다양한 인종, 문화, 종교가 공존하는 다문화 사회이다. 주요 도시로는 뉴욕, 로스앤젤레스, 시카고, 워싱턴DC 등이 있으며, 세계적으로 유명한 자유의 여신상, 화이트하우스, 할리우드 등이 있다. 미국은 세계 최고의 경제 대국이며, 과학기술, 문화, 엔터테인먼트 분야에서도 선두를 달리고 있다.
CPU times: user 35.5 ms, sys: 6.1 ms, total: 41.6 ms
Wall time: 3.05 s


In [11]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "미국"})
print(response.content)

미국은 북아메리카에 위치한 연방 공화국으로, 50개의 주로 이루어져 있다. 세계에서 가장 강대한 국가 중 하나로 경제적, 군사적, 문화적으로 세계적 영향력을 가지고 있다. 원칙적으로 민주주의를 지향하며, 다양한 인종, 문화, 종교가 공존하는 다문화 사회이다. 주요 도시로는 뉴욕, 로스앤젤레스, 시카고, 워싱턴DC 등이 있으며, 세계적으로 유명한 자유의 여신상, 화이트하우스, 할리우드 등이 있다. 미국은 세계 최고의 경제 대국이며, 과학기술, 문화, 엔터테인먼트 분야에서도 선두를 달리고 있다.
CPU times: user 8.79 ms, sys: 1.4 ms, total: 10.2 ms
Wall time: 12.9 ms
